In [ ]:
%pip install snowflake-snowpark-python


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# from snowflake.snowpark import Session

# # Snowflake connection parameters
# connection_parameters = {
#     "account": "your_account",
#     "user": "your_user",
#     "password": "your_password",
#     "role": "your_role",
#     "warehouse": "COMPUTE_WH",
#     "database": "MOVIES_DB",
#     "schema": "PUBLIC"
# }

# # Establish Snowflake session
# session = Session.builder.configs(connection_parameters).create()


In [ ]:
# Load the raw_movie_bookings table
raw_movie_bookings = session.table("raw_movie_bookings")

# Show a preview of the data
raw_movie_bookings.show()

In [ ]:
from snowflake.snowpark.functions import col, lit, when

# Filter completed bookings
completed_bookings = raw_movie_bookings.filter(col("status") == "COMPLETED")

# Add a total_price column and high_value flag
transformed_df = completed_bookings.with_column(
    "total_price", col("ticket_count") * col("ticket_price")
).with_column(
    "high_value", when(col("total_price") > 50, lit(True)).otherwise(lit(False))
)

# Show transformed data
transformed_df.show()

In [ ]:
# Save transformed data to a new table
transformed_df.write.save_as_table("high_value_bookings", mode="overwrite")

In [ ]:
from snowflake.snowpark.functions import sum, avg

# Aggregate data
aggregated_df = raw_movie_bookings.group_by("movie_id").agg(
    sum(col("ticket_count")).alias("total_tickets"),
    avg(col("ticket_price")).alias("avg_ticket_price"),
    sum((col("ticket_count") * col("ticket_price"))).alias("total_revenue")
)

# Show aggregated data
aggregated_df.show()

In [ ]:
from snowflake.snowpark.types import IntegerType, StringType
from snowflake.snowpark.functions import udf

# Define a UDF to classify movies
@udf(return_type=StringType(), input_types=[IntegerType()])
def classify_revenue(total_revenue):
    if total_revenue > 100:
        return "Blockbuster"
    elif total_revenue > 50:
        return "Hit"
    else:
        return "Average"

# Apply the UDF
classified_df = aggregated_df.with_column(
    "movie_category", classify_revenue(col("total_revenue"))
)

# Show classified data
classified_df.show()